In [29]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from edward.models import Normal
from edward.models import Gamma
from edward.models import GammaWithSoftplusConcentrationRate
plt.style.use('ggplot')

In [2]:
import numpy as np
import h5py as h5
import os
import re

In [43]:
th5f="/home/nwknoblauch/Desktop/scratch/polyg_scratch/RSSp_genome_direct_quh_chunk/bigsub_100_sim.h5"
hf = h5.File(th5f,'r')
quh_mat=hf['/100/quh'][:,:]
print(quh_mat.shape)
Dd=hf['/100/D'][:]
print(D.shape)

(202, 108)
(202, 1)


In [46]:
P=202
G=108
D=tf.placeholder(tf.float32,[P])
Q=tf.placeholder(tf.float32,[P])
sig_u=Gamma(concentration=tf.constant([2.0]),rate=tf.constant([0.0]))
c=Gamma(concentration=tf.constant([2.0]),rate=tf.constant([0.0]))
dm=((sig_u*D)/(c+D+sig_u))*Q
dv=(sig_u*(c+D))/(c+D*D*sig_u+D)
U=Normal(loc=dm,scale=dv)

In [42]:
Ql=[quh_mat[:,i] for i in range(quh_mat.shape[1])]
Qld=Ql[0]

In [47]:
qc=GammaWithSoftplusConcentrationRate(tf.Variable([1.0]),tf.Variable([1.0]))
qsig_u=GammaWithSoftplusConcentrationRate(tf.Variable([1.0]),tf.Variable([1.0]))
inference=ed.KLqp({c:qc,sig_u:qsig_u},
                  data={Q:Qld,D:Dd})

In [48]:
inference.run(n_samples=5, n_iter=250)

250/250 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan


In [49]:
Q_post = ed.copy(U, {c:qc,sig_u:qsig_u})


Mean squared error on test data:


KeyError: 'User must specify output_key.'

In [26]:
def build_toy_dataset(N, w):
  D = len(w)
  x = np.random.normal(0.0, 2.0, size=(N, D))
  y = np.dot(x, w) + np.random.normal(0.0, 0.01, size=N)
  return x, y


#ed.set_seed(42)

N = 40  # number of data points
D = 10  # number of features

w_true = np.random.randn(D) * 0.5
X_train, y_train = build_toy_dataset(N, w_true)
X_test, y_test = build_toy_dataset(N, w_true)


In [38]:
y_test

array([ 4.36791871, -5.34141151,  4.97126687, -3.12563648, -2.56821187,
        2.99361081, -0.54057834,  0.59667805, -2.30414473,  2.87368786,
       -4.18721493, -1.1964148 , -2.40054934,  1.28599195,  4.50576373,
        6.26784077, -3.81661083, -1.19707556, -5.84803546, -1.87125032,
       -0.23099218, -1.31508337, -5.55036602, -3.30120095, -2.04541092,
        5.23222258,  0.33729806,  1.19653078,  2.36060521, -3.61170066,
        1.19600324, -5.06737142,  0.43540289, -1.17156695,  3.02860869,
        1.07497101, -3.49993053,  2.51042875,  5.24274763,  2.20599875])

In [27]:
X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

qw = Normal(loc=tf.Variable(tf.random_normal([D])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
qb = Normal(loc=tf.Variable(tf.random_normal([1])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [28]:
inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(n_samples=5, n_iter=250)

250/250 [100%] ██████████████████████████████ Elapsed: 3s | Loss: 65.544
